In [ ]:

# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')




Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!fusermount -u drive
!google-drive-ocamlfuse drive

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

fusermount: failed to unmount /content/drive: No such file or directory
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
# below is the code for the inception block
from torch.nn.modules.batchnorm import BatchNorm2d
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()

        assert out_channels%8==0, 'out channel should be mutiplier of 8'

        ################################
        ## P4.1. Write your code here ##
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels,out_channels//4,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//4),
            nn.ReLU()
        )
        self.block_2= nn.Sequential(
            nn.Conv2d(in_channels,out_channels//2,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//2),
            nn.ReLU(),
            nn.Conv2d(out_channels//2,out_channels//2, stride=1,padding=1,kernel_size=3,bias=False),
            BatchNorm2d(out_channels//2),
            nn.ReLU()
        )
        self.block_3 = nn.Sequential(
            nn.Conv2d(in_channels,out_channels//8,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//8),
            nn.ReLU(),
            nn.Conv2d(out_channels//8,out_channels//8, stride=1,padding=2,kernel_size=5,bias=False),
            BatchNorm2d(out_channels//8),
            nn.ReLU()
        )
        self.max_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
            nn.Conv2d(in_channels,out_channels//8,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//8),
            nn.ReLU()
        )


        ################################

    def forward(self, x):

        ################################
        ## P4.2. Write your code here ##
        output1= self.block_1(x)
        output2= self.block_2(x)
        output3= self.block_3(x)
        output4= self.max_pool(x)
        # print("output1 size {}, output2 size {}, output3 size {}, output4 size {}, ".format(output1.size(),output2.size(),output3.size(),output4.size()))
        output = torch.cat( (self.block_1(x),self.block_2(x),self.block_3(x),self.max_pool(x)), 1 )


 
        ################################
        return output

In [ ]:
# Below is our code for the inception model, which consists of inception blocks defined above and other layers

from torch.nn.modules.flatten import Flatten
from torch.nn.modules.pooling import MaxPool2d
class MyNetwork(nn.Module):
    def __init__(self, nf, block_type='inception', num_blocks=[1, 1, 1]):
        super(MyNetwork, self).__init__()

        
        self.block_type = block_type

        # Define blocks according to block_type
        if self.block_type == 'inception':
            block = InceptionBlock
            block_args = lambda x: (x, x)

        # Define block layer by stacking multiple blocks. 
        self.block1 = nn.Sequential(*[block(*block_args(nf)) for _ in range(num_blocks[0])])
        self.block2 = nn.Sequential(*[block(*block_args(nf*2)) for _ in range(num_blocks[1])])
        self.block3 = nn.Sequential(*[block(*block_args(nf*4)) for _ in range(num_blocks[2])])

        ################################
        self.pre_block1 = nn.Sequential(
            nn.Conv2d(3,nf,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.pre_block2 = nn.Sequential(
            nn.Conv2d(nf*1,nf*2,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.pre_block3 = nn.Sequential(
            nn.Conv2d(nf*2,nf*4,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.final = nn.Sequential(
            nn.AdaptiveAvgPool2d( 1 ),
            nn.Flatten(),
            nn.Linear( nf*4 ,10 )
        )


        ################################

    def forward(self, x):
        # the forward function
        #######################################################################

        output1_1=self.pre_block1(x)
        output1= self.block1(output1_1)
        output2=self.block2(self.pre_block2(output1))
        output3=self.block3(self.pre_block3(output2))
        output = self.final(output3)
        # print("size of tensor after final stage: ",output.size())

 
        #######################################################################
        return output

In [ ]:
# Configurations & Hyper-parameters

from easydict import EasyDict as edict
device = ("cuda" if torch.cuda.is_available() else "cpu")

# set manual seeds 
torch.manual_seed(470)
torch.cuda.manual_seed(470)

args = edict()

# basic options 
args.name = 'main'                   # experiment name.
args.ckpt_dir = 'ckpts'              # checkpoint directory name.
args.ckpt_iter = 1000                # how frequently checkpoints are saved.
args.ckpt_reload = 'best'            # which checkpoint to re-load.
args.gpu = True                      # whether or not to use gpu. 

# network options
args.num_filters = 16                # number of output channels in the first nn.Conv2d module in MyNetwork.
args.block_type = 'inception'              # type of block. ('mlp' | 'conv' | 'resPlain' | 'resBottleneck' | 'inception').
args.num_blocks = [5, 5, 5]          # number of blocks in each Layer.

# data options
args.dataroot = 'dataset/cifar10'    # where CIFAR10 images exist.
args.batch_size = 32                # number of mini-batch size.

# training options
args.epoch = 40                     # training epoch.

# tensorboard options
args.tensorboard = True              # whether or not to use tensorboard logging.
args.log_dir = 'logs'                # to which tensorboard logs will be saved.
args.log_iter = 100                  # how frequently logs are saved.

In [ ]:
# Below is our final model which uses the inception model, loads the weights after training on Cifar10
# we then freeze the layers so we do not retrain them, afterwards we add a linear layer.

from torch.nn.modules.flatten import Flatten
from torch.nn.modules.pooling import MaxPool2d
class newnetwork(nn.Module):
    def __init__(self, num_classes=3):
        super(newnetwork, self).__init__()
        """Initialize an entire network module components."""
        self.num_classes=num_classes
        self.inception_model =  MyNetwork(args.num_filters, 'inception', args.num_blocks).to(device)
        self.inception_model.load_state_dict(torch.load('/gdrive/MyDrive/project/car_damaged_or_not/inception_pretrained.pt'))
        for param in self.inception_model.parameters():
          param.requires_grad = False
        self.dropout = nn.Dropout(0.3)
        self.lin = nn.Linear(10,num_classes)
    def forward(self,x):
      self.inception_model.eval()
      with torch.no_grad():
        inception_output = self.inception_model(x)
      return self.lin(self.dropout(inception_output))
      




In [ ]:
# check which layers are frozen
def dfs_freeze(model):
    for param in model.parameters():
        print(param.requires_grad)


In [ ]:
new_model = newnetwork()


In [ ]:
dfs_freeze(new_model)

In [ ]:
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
# Basic settings
from os import path
from pathlib import Path


device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'

result_dir = Path('/gdrive/MyDrive/project/car_damage_location/self_inception')
result_dir.mkdir(parents=True, exist_ok=True)

global_step = 0
best_accuracy = 0.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from tqdm import tqdm


In [ ]:
# create my custom dataset using the csv file that we created in another notebook 
# csv file has the images and the 1-hot encoded vector
from torch.utils.data import Dataset
import pandas as pd
import numpy
import os
from PIL import Image
import torch

class Damage_location(Dataset):
    def __init__(self, root_dir_front,root_dir_side, root_dir_rear, annotation_file, transform):
        self.root_dir1 = root_dir_front
        self.root_dir2 = root_dir_rear
        self.root_dir3 = root_dir_side
        self.annotations = pd.read_csv('/gdrive/MyDrive/project/'+annotation_file)
        self._transform = transform

    def __len__(self):
        return len(self.annotations)

    # def __set_trans__(self, new_trans):
    #     self._transform = new_trans
        

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        # print(self.annotations.iloc[index, 1][1:-1])


        # print(self.annotations.iloc[index,1])
        value = numpy.fromstring(self.annotations.iloc[index, 1][1:-1], dtype=float, sep=' ')
        # print(value)
        value = torch.from_numpy(value)
        # print(value)
        # print(type(value))

        # print(value)
        if value[0] == 1:
          root_dir = self.root_dir1
        elif value[2] == 1:
          root_dir = self.root_dir2
        else:
          root_dir = self.root_dir3
        img = Image.open(os.path.join(root_dir, img_id)).convert("RGB")
        npp = numpy.fromstring(self.annotations.iloc[index, 1][1:-1],dtype=float,sep = ' ')
        y_label = torch.from_numpy((npp))

        if self._transform is not None:
            img = self._transform(img)

        return (img, y_label)

In [ ]:
root2= '/gdrive/MyDrive/project/car-damage-dataset/data2a'
training = root2+'/training'
# we then split the training in train, valid. so we store everything in the training folder
front = '/00-front/'
rear = '/01-rear/'
side = '/02-side/'


In [ ]:
num_epochs = 40
learning_rate = 0.04
batch_size = 33
shuffle = True
pin_memory = True
num_workers = 1


In [ ]:
# Define train/test data loaders  
# Use data augmentation in training set to mitigate overfitting. 
# Use data augumentation to increase the size of our training set and validation set
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),                                
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

center_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ColorJitter(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),                                
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

test_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(32),                       
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])



import copy

dataset = Damage_location(training+front,training+side,training+rear,"damage_locations_3.csv",transform=test_transform)
train_set, validation_set = torch.utils.data.random_split(dataset,[959,191])

test_set_2 = copy.deepcopy(validation_set)

train_set_2 = copy.deepcopy(train_set)
train_set_3 = copy.deepcopy(train_set)
train_set.dataset._transform = transform
train_set_2.dataset._transform = transform
train_set_3.dataset._transform = center_transform

# here we increase the dataset
increased_dataset = torch.utils.data.ConcatDataset([train_set_2,train_set,train_set_3])
new_test_dataset = torch.utils.data.ConcatDataset([validation_set,test_set_2])

train_loader = DataLoader(dataset=increased_dataset, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=new_test_dataset, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
# check which transform was used
print(train_set_3.dataset._transform)

Compose(
    Resize(size=128, interpolation=bilinear, max_size=None, antialias=None)
    ColorJitter(brightness=None, contrast=None, saturation=None, hue=None)
    RandomCrop(size=(32, 32), padding=4)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)


In [ ]:
def train_net(net, optimizer, scheduler, block_type, writer):
    global_step = 0
    best_accuracy = 0.45
    num_classes = 3
    train_samples = 959
    test_samples = 191

    for epoch in range(args.epoch):
        num_samples = 0
        # Here starts the train loop.
        net.train()
        for batch_idx, (x, y) in enumerate(train_loader):
            num_samples=0
            global_step += 1

            # Send variables to gpu or cpu, get output from model = logit
            x = x.to(device=device)
            y = y.to(device=device)            
            logit = net(x)

            # Compute accuracy of this batch using `logit`, and keep it in a variable called "ratio" and 'accuracy'.
            num_correct = 0
            t1 = torch.tensor([1.,0,0])
            t2 = torch.tensor([0,1.,0])
            t3 = torch.tensor([0,0,1.])
            prediction_list =[]
            for score in logit:
              max_index = score.argmax()
              if max_index == 0:
                prediction_list.append(t1)
              elif max_index ==1:
                prediction_list.append(t2)
              else:
                prediction_list.append(t3)
            # print(prediction_list)
            predictions = torch.stack(prediction_list).to(device)
            
            # print('prediction is:',predictions)
            # print('y is: ',y)

            # compute the number of right predictions, accuracy
            for sample in range(predictions.size()[0]):
              if torch.equal(predictions[sample],y[sample]):
                num_correct += 1
            num_samples += predictions.size(0)
            print('num of correct: ', num_correct, ' batch size of samples: ', num_samples)
            ratio = float(num_correct)/float(num_samples)
            accuracy = ratio

            # Compute loss using `logit` and `y`, and keep it in a variable called `loss`.
            # print(logit[0:10],y[0:10])

            # crossentropy works well for multi-class
            loss = nn.CrossEntropyLoss()(logit, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if global_step % args.log_iter == 0 and writer is not None:
                # Log loss and accuracy values using `writer`. Use `global_step` as a timestamp for the log. 
                writer.add_scalar('train_loss', loss, global_step)
                writer.add_scalar('train_accuracy', accuracy, global_step)

            if global_step % args.ckpt_iter == 0: 
                # Save network weights in the directory specified by `ckpt_dir` directory. 
                torch.save(net.state_dict(), f'{ckpt_dir}/{global_step}.pt')
        # Here starts the test loop.
        num_samples = 0
        net.eval()
        with torch.no_grad():
            test_loss = 0.
            test_accuracy = 0.
            test_num_data = 0.
            for batch_idx, (x, y) in enumerate(validation_loader):

                # Send variables to gpu or cpu, get output from model = logit
                x = x.to(device=device)
                y = y.to(device=device)

                # Feed `x` into the network, get an output, and keep it in a variable called `logit`.
                logit = net(x)
                t1 = torch.tensor([1.,0,0])
                t2 = torch.tensor([0,1.,0])
                t3 = torch.tensor([0,0,1.])
                prediction_list =[]
                for score in logit:
                  max_index = score.argmax()
                  if max_index == 0:
                    prediction_list.append(t1)
                  elif max_index ==1:
                    prediction_list.append(t2)
                  else:
                    prediction_list.append(t3)
                # print(prediction_list)
                predictions = torch.stack(prediction_list).to(device)

                num_correct = 0

                for sample in range(predictions.size()[0]):
                  if torch.equal(predictions[sample],y[sample]):
                    num_correct += 1
                num_samples += predictions.size(0)
                print('num of correct: ', num_correct, ' num of samples: ', num_samples)
                ratio = float(num_correct)/float(num_samples)             
                accuracy = ratio
               # Compute loss using `logit` and `y`, and keep it in a variable called `loss`.
                loss = nn.CrossEntropyLoss()(logit, y)
                # Compute accuracy of this batch using `logit`, and keep it in a variable called 'test_accuracy'.
                test_loss += loss.item()*x.shape[0]
                test_accuracy += num_correct
                test_num_data += x.shape[0]
                print('test_num_data: ',test_num_data," num_of_samples: ", num_samples)

            test_loss /= test_num_data
            print('\n ', test_accuracy , test_num_data ,'\n')
            test_accuracy /= test_num_data
            # checkpoint
            if writer is not None: 
                # Log loss and accuracy values using `writer`. Use `global_step` as a timestamp for the log. 
                writer.add_scalar('test_loss', test_loss, global_step)
                writer.add_scalar('test_accuracy', test_accuracy, global_step)

                # Just for checking progress
                print(f'Test result of epoch {epoch}/{args.epoch} || loss : {test_loss:.3f} acc : {test_accuracy:.3f} ')

                writer.flush()

            # Whenever `test_accuracy` is greater than `best_accuracy`, save network weights with the filename 'best.pt' in the directory specified by `ckpt_dir`.
            global max_accuracy
            if test_accuracy > max_accuracy:
                max_accuracy = test_accuracy
                torch.save(net.state_dict(), f'{ckpt_dir}/{block_type}_best.pt')
    
        scheduler.step()
    return best_accuracy


In [ ]:
# Function for weight initialization.
# We avoid initializing for the frozen layers
def weight_init(m):
    if (isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d)) and m.weight.data.requires_grad:
        torch.nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)

In [ ]:
# List of all block types we will use.
global max_accuracy
# Only create a new best.pt file if our accuracy is better than in the last one.
max_accuracy = 0.47
block_types = ['inception']

# Create directory name.
num_trial=0
parent_dir = result_dir / f'trial_{num_trial}'
while parent_dir.is_dir():
    num_trial = int(parent_dir.name.replace('trial_',''))
    parent_dir = result_dir / f'trial_{num_trial+1}'
print(f'Logs and ckpts will be saved in : {parent_dir}')

# Define networks
networks = []
network = newnetwork( num_classes=3).to(device)
network.apply(weight_init)  
networks.append(network)


Logs and ckpts will be saved in : /gdrive/MyDrive/project/car_damage_location/self_inception/trial_9


In [ ]:
print(max_accuracy)

0.47


In [ ]:
# Setup tensorboard.
if args.tensorboard:
    from torch.utils.tensorboard import SummaryWriter 
    %load_ext tensorboard
    %tensorboard --logdir "/gdrive/My Drive/{str(result_dir).replace('/gdrive/My Drive/', '')}"
else:
    writer = None

In [ ]:
final_accs = {}

# Training the model 
for block_type, net in zip(block_types, networks):
    try:
        args.name = block_type

        # Define optimizer
        optimizer = optim.SGD(net.parameters(), lr=0.015, momentum=0.9, weight_decay=0.0001)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,80], gamma=0.5)
        
        # Create directories for logs and ckechpoints.
        ckpt_dir = parent_dir / args.name / args.ckpt_dir
        ckpt_dir.mkdir(parents=True, exist_ok=True)
        log_dir = parent_dir / args.name / args.log_dir
        log_dir.mkdir(parents=True, exist_ok=True)

        # Create tensorboard writer,
        if args.tensorboard: 
            writer = SummaryWriter(log_dir)

        # Call the train & test function.
        t1 = time.time()
        accuracy = train_net(net, optimizer, scheduler, block_type, writer)
        t = time.time()-t1
        print(f'Best test accuracy of {block_type} network : {accuracy:.3f} took {t:.3f} secs')
        final_accs[f'{block_type}'] = accuracy*100
    except Exception as e:
        print(e)

# Print final best accuracies of the models.
for key in final_accs.keys():
    print(f'Best accuracy of {key} = {final_accs[key]:.2f}%')


In [ ]:
import numpy

In [ ]:
print(max_accuracy)

0.4712041884816754


In [ ]:
img =Image.open('/gdrive/MyDrive/project/car-damage-dataset/data2a/training/02-side/0257.jpeg')

In [ ]:
img.show()

In [ ]:
def preprocess(img):
  img = transform(img)

  return img


In [ ]:
# test
import urllib

from urllib.request import Request, urlretrieve  # Python 3
model_loaded=model

def test(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = transform(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)


    with torch.no_grad():
      output = model_loaded(img)
      print(output)
      # print(output)
      if output[0].argmax()==0:
        return ' Front damage '
      elif output[0].argmax()==1:
        return ' Prediction is: Side damage '
      else:
        return ' Prediction is: Rear damage '





In [ ]:
url='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFhYZGBgaGhgYGhwcGhkcIxoaGhoaGhgaGhocIS4lHh4rIRoaJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHhISHzQrJCs0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAKgBLAMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAADAAECBAUGBwj/xABCEAACAQIDBQUGBAUBBwUBAAABAgADEQQhMQUSQVFhInGBkaEGEzJCUsEUsdHwM2JykuEVByOCorLC8SQ0RFNUFv/EABoBAAMBAQEBAAAAAAAAAAAAAAABAgMEBQb/xAAnEQACAgICAgEEAwEBAAAAAAAAAQIRAzESIQRBUTJxkbETYYHBIv/aAAwDAQACEQMRAD8AoLSk1pyzuAakRt9R8wjckj6NWCFOSakLZ2t1jtXQcT+UGMWn039ZnLKvRahJ+gfu0Gm7foo/SRdCflPjlD/j1+g+dvyEj+MH0L4m8wk0/ZrGMvgrLh2Om6O65/KHp4F+THuAH5kmM2Pb6go5DL0Eh+POgY+F5Fw9mnGb1RaGzn1CAf1NDUtlvxZR3XMzGxr8LjqdfM3Mi+KPzMzeZ/ODlD4Y/wCLI/a/BqthUTV7+X5kwFVF6W6/4mf+KPAeZvGUljmZm3elRccMl22X1qoDm1uigC/jDbyN8KsfM+ZlrZuCQi4TPmwJ/PITXTCn9/4iRzzyRi6MSjg2J+A25Xt66esvrswnREXrYk/pNamN3uhSw1lI5pZpPRkpsJfmck9LW8tIJ/Z9PrY+X5ATYbEKOXnI/iB/5BH52iqJKy5V7M+jsJPmLH0/zI4zYaWO6BLtTFJz8oF8co5+Ji6HGWW7tnK4rClDlD4NnPyn9982C4bMJfrYmMQ/BPyEk6nmbjTQFKZ4yttNQqo9s0rUTnyLhD/ys0tMX47o8bzM2y5NGqLkkKWFlsLqN4a5nMSo7RjNNxZqLht266i5senCP7kfu0K1Byu+LbpzGuh0gAzcge4yQTcu7HNIfsSPu+vqZJawPQ9Y7VANTGDtAvdnn6iMyHn5iEFRTyMkBygFv2VKiAizKD4CVW2dTOmUu16Ln4cu60rFHX4gD6Qs1jJ+mU6myB8rSpU2a68L901BXPFSO7OHdwq7zNYfn3COzT+SS2YCYWppZh4SwuGQaXc8z8I7ucLVxRfIZL1Ovf8ApIVMSFy1PIRhKcmV66cz/juA0lfdTmY1VixzP6SG6nONIRdd/wCaR95+zBmt3+FhIZmK2dUYFjx/fjGLn6h4QK02bQE+BhPwT67h8jF/pSjFbY/vB3xDfOmUtbOwSs9nJt0nW4bD4dB2VW/O1z5xUY5c8cfSVs5Cjs520BPh+s18L7PObbzbo6Zmb/4gaqjHuU285FsW9slRf63X/tvHS9nLLycktJIy8V7O2HYJJ6/paYGJwhQkETqziKj5e8Hcin/qbKBOEBN2I/4jvHwUZQ+xWLPKP1OzlVpk6AnuBMv0tnsc7Ed+U6KnheSu3cu6JY/DsPlRerMCfWHZU/L+Cjs4uot8Xdcy69R+JC97fYZx/d3+KsLfy6ekIuGoDVnbuyhRyTmm7a/CKvvANWJ7hb1P6SHvAdFLHqSfQWmgKtFfhpbx/mN/1k/9RYDsqqjuhS+SeT9RM9KVVvhRh4bvrI4nDMvxHM8BmZdqbQc6t4DKUalZr6/eJ0VFzvuh6eFVvibdHU/YQ71MOg4E8zMyspOZLH0lCvRB1NojVY+T7bNLEbeGi5D+UfeZtfa7HRSe8ys2FvpeRXAHnHZvHHiiEO0n/lHhAYisahFz0y5HIyymzl4tDGiiC9wTCwbgtIVPabe4oXA/hp57i39YqG0iSBuXJIAtqb6WHEwOBF6Kg2spqJ/bUZR6ASxs5RRrJUtvBWBI525deI6iU/qMocVj13X5O2wPs0HCviBum3wKbf3MOPRfMzdw+z6VPJKar3KPzOZ85QT2nwxF/eW6FWv+Uz9pe2Cqp90hc8Ceyvmc/SatqLVPo8WUPIyypp/b0F9q8PSKAsqh94BSBY9Rcai32nN/gG+VwekxMbtKtWffcne0AGQUclH7Jk6W0HXVTM5y5StHqYfGnjglfZqNQqLqt+6Q96w1U+UrDb7jgZCrt9yLKLdSJJaxz9pBMdiwguy58LgXM53EV2c7zHu5AR8TXLkknebmdBA+5ZsyfP8ASNI3ikkDNQ8PSAL2uSbRY7EqgzP75Cc1jcS9Q20XgL/nNoY3I5/I8yGFVt/Bfx22QOymfU6eHOYz4hiblj5yLUmHAwc6FBLR4WXycmR22epf6O4y/IEw1D2eqMePgCZvIrcvM/aSYHi1u7/M86z6N+RP1RDD7JdQMiPT0vDvRc5bvjBrT/qMIARxt4wMHKTdtgF2e17hWv3CWBgap1G6OrWjNW5sfC8j7zpfvh0Dc2EGzx81VR3Ek+gk1oUhqWbuUD1Ykyt7w9BFv8zHaJak9stmugyFMd7MW9NINcaw+EKO5VECo5CTFPnC2LjFbGbFM2rN5/pIFBqfWFK2kgQIUO0tEAgMkE6SJrcBH3ucOhdiYHnGfIayq9c31khUJ1hZfFjtU5yDOD0kWQnjIEqNTEUkiFUHnK7C0sVK+WQlZl4sfCBpEg9cjUwZLNoDCNUUcoF8VGikvhBBQ5mRbDJzzld8TIfiDGkPjItbOPYdR8tVx/duv/3wvu+bTNw2Jsao5uD500H2hDXMctkY4yr8/sve7/mjqjDQgyh74wgxR5SaNOLLVzxH3jEnl6Sv+LjHFxiphncAZjLxmfWq3/SQxOKLZQG6T0jSDQmrW0lPHY0oM9ToOJ/QQ2NxK0lva7H4R15npMLcZySxJZv34TfFivt6PP8AM8xYlxjv9AXJc7zG/wBugljD0VtckSzQwhtew1sb+UtKgAse493A3nYkkeFKTk7b7KPu11+0KMIhzsvlLiFTkTzGX1DMGHp1wQNL6HIajWFCO6NROpje++lQPC8Eq9JJKZ/Znkn01IRrMef5RySeUmKMQy0EbC16Bt1MZbnQWEKY6GILIOg4xrDlCOJBEjoL6JqYVWvAOto9ExCa6sOxg3cQFZ7QTNzjscYhPeWOUhUrHnA7jHoOsXuVGpuYjRJIGAWOUtC4EA+JVRqBKFbaY+XOBXGUtI0KhPOVmdVzJzma+IduNoPd5m8dGkcdbZefaA4Zyu+IY9IMWg3eNIpRS0iTPBNVgqlSBarCi1EOWiDyr7yIVJVBRboN2n/ppn1cfYQnvJRSpYseaqPJmP3jmpG12Rjj0/u/2XfeR/eSh72OGJk8S+Jc97IvUJyHjK63OQBJ6SwiWyOXQC5lKJnOaiMFtAYrFKgvx4DmY+KdlF9xgPqKkDz0nP18Tvt+X6zSGO32ed5fkrFG1t6JO5ckvqfTlaFXEWAyuZVVOJMOotO1KjwJScnbCnEOe48InxJORMGWFvvItVGmX76xiCrzzj5dfT9ZWbEHhI++6HzgB66RFTjtGWeUj6QIsaoI4iIjJIcIhJbpkfdnnIKscxCw1MgzAcYM1Bw8/wDMdjSsNUqLylUsxORsOkFVqWzJAHWVau0l4XbuiNYwfpF/ePFvSDaqB+syKm0HOgtKzbxNyYUaxwv2zUr7RUZAknpKL4x20yEFkJFq0dGsYRWkSNPiTePcSuasi1TnHRVFgtBu8rPiRwgHqk9I0h8WWnr9YBq/KBDSdOi7aAmWkJ1HZBmJ1jTTo7HNru6IP7j6ZQ+JwlFVshLtzNwB1ux9N098fFmTzxul2YtpOnSJIAFyZP3fW/d/mXMIifMXv8qqqm55Es2ncM40gnkpWhsVs4IUBcMWVt5QM0IIsGz1OetpVNLOwz7s/ObW1Sy+63qQQbrlPiByZLkl8ze+lzochKbuTkFA8L/nHJI58OWTTb+WU/cW1y/OLfGli0LVpnibn8vKDWlzvbyv4mSb8rVtlvD4gjKyoO+1+/me+aODxDqwCPRQ6mw3yP6lQ73hMkbg1APCxI/NyLf4hMTtZ6KFwNwjspukjtnP4lyytfI8JrBdnD5Ekk2ZXtXtJ6lVlZ2axsbgqLgDJUIBUdLXmKqyL1LksSWYkliTcsSbkknUk53kWcmdMUkjwck3OVsMzgfpIvVJyg7TQ2fsavWF6dNiv1nsp/e1h5RmZRZiY06Wn7MIv8bEKD9KDe/5jYQ6YXBpojVD/O5z8FtADk7xT1XYOwKdUb74akicOyST4kzc/wD5bB//AJ6f9sLHRSXSLdMgXkHqzykz6OmWAIi4EptX5mCevHY+DZdavBVMVM6piJXeuTJNI4i9UxAlWrimOmUqu8G1TrA3jBIm5vqb98GWEA9blBPVHOVRqoh2qiDatKzVhB++jUS6LDvBtVgczrCIo/ecqhtpDM5MgRCjpYRyvWOiOYIIT0k6dEc79JJVHLzhN+0CJSYWiig6cO7/ADCvXtoQv9I+5v8AaUjU/YkN/wAZSMZd7LRqdCev+T3QLMTqY6UWbgT5xF1U9o+AAHmYUTa9CpUieB/Kb2xsIjZl1U8lR2Y8NQhHheYIxqqclB7+1DU6zuCxfdUa57o7gBrKj0ZZOUlujc9q6y7+HDi4FOpu9oFiS1HeLjLd0y7phVcZ9K7vrBV6ikruj4VYE2IzLJfPQ6GBZjyjk7YsGNRjT+SYqGCqV78T+/KQdspXv1kI6HRYp1lByUnkL2HkoEzNtbRaqyp2QlMFUCgAXJu7XtdiTbM3vuiW8ehp09/O7HdTI2vqxDWsSBbTQssxETjwnRjXs8Xz8ybUF/olWbWwfZ6vi2IpJ2QbPUbJE6FuLfyi5nQeyHsQawWtiQyUTmiDsvVHAk6onXU8LDOen06aoioiqiKLKiiyqOQE1Z5uzzTEbOw+Ayan76qLXeqBuA69ilp3FyT3Tntqe2FSodS3LgB3DQDuE9M9tNjnEYdt341U+XA+E8RxNMJlqf3wiuwoLU2rUY5nLlPQvYXAUq6CpY7wJVgc8xbQ8sxPLyZ7h/sp2buYVHYfGWfwJO76AGUtikda6BVVRlYXg96ExD3YmVi8hstdI4968C+ImQ2LPCCJZtTPNo+sWI0qmOEC2JY9B1lNWA0zjMQdY6LUEg/vx/V1kGxBOQEAXtAVK4EKLUUEeub5Z9TpIviCJWNS8iZXEukSZyTcm8QYyIj3lUVY+7zivGUQiLGQ5iX9mSXzhaVItkoJ7h94Q0QDY68hmfSOjKWRATI79ppUdmu2iHx+/KWjhKKD/eOL/SnaPmco1FmLzpdbMUFzoss4bZtV9AQOfDxJlyrtmmgtSpD+pzfxAmVjNsVX+JyByXsj0j4xRPKctKvuaf8ApdJP4tZV6L2j+kj+Pwyfw6bORxY7o9M/OYJfzkWaF1oTxt/U2zRxW1HbK4VfpQW9dfWUVYk9585C009kUaZe9RwgBB0JvnoLQ7YNqEekW9mez7v23Komtywzz5AGXdo4FEAANwBbQjvPaN9Pzm22LpblqZUDhYW5/KM5k4kjPfPUfKNbcBymnFJHCss5St9f0YOIobuYGR3RwyvvHPO4+A+UAtxrcDzlraGI7ZsALIg0Fs98nPrKTNeRI7MTbVsaqd7S/DU+tgITDbwYWtfK2V/vnBU9c/zt+s0mxjUaDuqIDbdVirFt5rgFd5bX1bX5chHGNsnPk4wbML2i2i1arYuWSmNxMgoFviKqMgCw8Qomv7P7Kp00XE4hQ186FJtH5Vag+jkvza6TO9l9lLUY1Kg/3NK28PrfVU7uLdMuMh7TbdapVIU3IsDy5BQBwE6F0fOSbk22ev7M2sK672jfMPuOkvB5417O7edKihmsfSes7OxYqpvLqPiXiO7mOsGvaBM0Ee2f7txnkn+072ZNCoMQg/3TntW+Vz9jPVlMJisCmIoPRqC6sCp+xHURIGfOWy8C1aqlJdXYL3D5j4C5n0ts/CijQRFFrKAByAFpyvsd/s8p4dzUZmdwSFJAAVb8ANSec63HVc7DhlLapELtlQqSZn7R2zSoPuORvWBOfP8A8TSq44KvZHatqeHdPFfavZ2KxGKqOKNQqDuId05qvEdCbnxkpWU2WQ5iL8b3lTf8oxfrPPo+1LjVrQJxErl5C8dAGZ7yEHvR7yuIWELRt6RtHVbx0LkODJqsdVA1kd7pEQ5WEDqOEsUagOi7x/tUd55SjuX1h0dBa936DIeJjSJkauHBeyly38lIZeJ4/vOXXdKQz3EPK5c+Nja/iZg1NpORuqdxOSZX7zqZRLky7Of+Jy26Rs4zbTNkCxHXsjwC2mS9Yk3kI6rfU2HOxP5cYtmsYxiukRJiC3l+klNbncLZZFyFF8vkU3tr8x7pKrtFt0qrhR9NNAg8W+KFEub9IzmUx0S5zyH74RXuYZVt/mIHIuVKtAJuU0Zn+ao51P8AKguByzlSknaGgzjGsIShRZzlK7MtItYbHGmW3SLnpc+H/mCFWo5yDMx03QSfIZzQwOBpg3Y73IKL+uk3MNtanTFgqD+pr/8AKth5ykvkwnkSf/mNs5Wlseu7sAliu6G3mUWNt4AgnWzA+M0sN7Mtft1aa9A1z6AwdXbQNSoy3BaoTZFQfCiUxbeDWFk05kyptDa72+Fhe+bu5NuiqQts+UGlYRlllFaVhdq4WlRBC9thlmSPEA6jwE56u9Ss6UVsSzdkWA7TZXNuAFzc8I7OSSTqTe81PZajc1MSwzuaad5ALkdwIXxMuC7s5/Nlxgot22WPaDFrhqC0aZ+EboP1Mfjc9SbmcKrceM9F2l7J1a9FnPZckMgPIXFjyvf0E5BvZbF3t7lj1BU/ea8XR5Foyd/jxnoPsXt1msN61RB/cv3nLU/ZLGH/AOO3iV/WdZ7Gew2IFX3lTsbo7K67xP1HgI1FibR6Zgtoo6gutm5jQ+HCF9+L3Gky6ey3pXD2GptfQd8WFxVN2KrURyNVVgT6RUVZvrtA7tgJze3dsGmrutN6u5cvulQFy3rXYi7WIO6tzYjnNUvOWT2drPUclwyM7sDdbAOSSDnllGlYm6NTYmOXEoHIKjip1HHhOsw9OlujSc/Sw6U1CJoNT9R426cIxxdsotaGu12eO3jFpG8cGcJ9lyFeKKPKFyFaOojXjFoC5BLxw0DeOGgS2FLyJeQAMkEgKxt6OTEwtIl4BY4EnTQnQEnoMutzGp7osTunoc/RfuYf8ao+XfPANko/4F+LxMdEOT9BcHgS5sLsRqEG9bvb4V9ZOvTRPidWb6VJe1uZWw8LypX2m7jdZjufQtkXxVdfGDohW+JwgF8gpY8NBkPNo+jNqT7k/wACr1mY3N7cMgB4WAEfD4Z3NkRnPcftLqYmgh7FJnb6qrE59KaWA8WaWq2KxRTX3KclC0l8wBfzMaiS5tdJV9wa7GcfxHSivU2P9o7XnaBfD0V+dn62sD3XNvUyhWKg5Nvnic/zgyxOdo+hJSe2XHdeGQk0xNrBRc9fteUlRjqbd8KjqCNTbqFH6+sBtFq1Vjul7Wyyz05zS2fsIMd4h3IBJ1PnYZeMyV2s6/AEU/VbeN+Y3r2gMTtCq6tvu7EjdALG2eQy01tBUZy5KLa6NXB0huqzVqVJWG/qrPZu1oLkHtcpWqVcKhJAeufqbsgnxzPiJkP004d3CQJhYKDSSb/4T2jigQWCquVrKLD96z1P2Q2ZRo0KPvFZnCBiMrBn7bG3E3NvATyhKO/UppqHdEP/ABML+l57NvWmsVSPI8ufLI0tLo3fx9I6o2X9MmNoUR8jeSn7zA97JLVJyAvNOUjjpHQptbD/ANJ/mBH+IdtoINBOcemoF6jKo5E5+Uq/i0GSnLhBtjpFf2y2i4elZN9GL74IDKSFsgYHUXO9b+XjpMbZeziaj4lgEVbM5Cqo7ItYBRbeOQsJ1FOqeNiDwIBHkYLagapTZAQMuyLBVBBB0GQva0a0DRzeP2k9RjmQvACNsuu4qJYnNgCOY4+mfhEuyaxNhTbvyt/de0vPsh0ouEINdhYEZhRftKDxYi4vAZaxm36VN1R98F81O4+6RxIYixGfC8vWBzmTs/YLuwqYm4Aue1e5va4UHM6d03xVUZAZQaoSdnjEVpDfivOE+vsJeMWkLx7GBNj3ivEEhUSAciAWEVIVEkmW/SKxOQJyB1MG7t3Q1gJXrOIISYMvG95IxSh2Lei3oo4jE5DWluiaSntB38QgPfkxt5RsFhnquKdNSzHgATpqeyCQOs0f9ERP/cYlEPFE/wB6/dZTurl9RHdGkZSyJdNgU2s6fwkSn1RLt/e+83kRGpYCvXa53m5lt5iP08bS1/qWGpfwcMHb68Q2+e/3a2QesobQ2xWrZPUYr9AsqjlZFsPSO0Zq2+lX9vZafBUqfx1V3vpTtt3ZdhfEmUqmKX5E8WO8fIZD1lMRzFZoovbZJnJ1N5EtIkxt6FA5JEi0bfzA63/tBI9QIMmQpm5Y8ALDvOf2jSMZzuo/IcmSdAoBfjouhPU8hBBwGF8wDc9bSFWqXYu2pNz/AI6TSEVs4vMzu+Ef9L2yKpOJw4NgBVUgDIC153O0vaPcBK0i1uJYAel551hau5VpPwV0Y928L+l53W3cMdxgovrl3TQ8453Ee3mIPw00XvDN+ZEz6/tljWy97ujkqgTHqUW+YFT/ADZcOvGPgsK9VtxFLH0HUngIuw6NLB7UxVVwBUck8gufpPUdj+zFYItV3N7DJuJ42HKG9hfYtKCCrVALW7r9B0m/tXa6g7oOmVhnb9JdUiLtlP3JAznK7T2mXqVaDUyQpVRcNYgrdiLEC9zqdLTfxOPVVLMd0AXJOVh1mNs/2nSo10RXANg7IpItyJF7QQ2WNg7LehTG8TZgLKc9NW/fKaLtYXOQhBiS3aY3J4mZntLX/wDTViP/AK3/AOkwBGVV9scOXKBwbG187HuJyMvJthSL3nkGGwzOQFFyZ6Fsj2DDUlZ8S9NjqoBsPWJWD6OZSoACLa/pl65+AhVqryOZudOd7d0UU5D6sdXBHEdRYXytny9eMOri5Ntb2yHG1vv5xRRMlhKbDO44C2Q6X/WSdgRpbM/f9+HkopBJAkSu9UCKKNDiVXqkyMUUplCjhY8UYho4iigSJahAIBIB1AJANuY4yIMeKBKGvFFFA0QmYQZeKKUjGcmRJivFFGc8pMHWcgZa6SVJbADjqe8xooEx+psiWzPlHEUU2jo8vI28jBPnceE9L2RiRiMMjn4rbr/1rkfPXxiilMzND2ZwN61XshldLMCARx1B1mtsL2bp02aqyqiKcgAFB8BFFNF9Jm/qLO09qs/ZTsJpyJH2Ex61ZERnYhUUEkmKKQUeW+03tG+IYqt1pA5DTe6t06TW9hx2D/WfyEUUS2U9Hct2suHGVNpYctRqIPnRwO8gj7xRRkk9jeyaUigUZWzbiTxM7OhgUCgWEUU1SRB//9k='
test(url)

tensor([[-2.9454,  3.0895, -3.4813]], device='cuda:0')


' Prediction is: Side damage '